In [1]:
import re
import pandas as pd
import numpy as np
from numpy import dtype
pd.set_option('display.max_columns', None)

In [2]:
labels_names = ['ncodpers_y', 'ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1', 'ind_cder_fin_ult1', 
                'ind_cno_fin_ult1', 'ind_ctju_fin_ult1', 'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 
                'ind_ctpp_fin_ult1', 'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1', 
                'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1', 'ind_plan_fin_ult1', 
                'ind_pres_fin_ult1', 'ind_reca_fin_ult1', 'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 
                'ind_viv_fin_ult1', 'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1']

In [3]:
na_values=['NA', 'nan']
nrows = 1000
# Store labels as string because later we will concatinate them
dtype = {'fecha_dato':np.str, 'fecha_alta':np.str}

iter_csv = pd.read_csv('../data/train.csv.zip', na_values = na_values, skipinitialspace = True, 
                       iterator=True, chunksize=100000, dtype=dtype)

#corrupted = pd.concat([chunk[chunk['ncodpers'].between(660248, 660250)] for chunk in iter_csv]) #Small dataset could be used for testing
corrupted = pd.concat([chunk for chunk in iter_csv])

C:\dev\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2821: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
C:\dev\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2821: DtypeWarning: Columns (10,15) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
C:\dev\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2821: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
C:\dev\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2821: DtypeWarning: Columns (10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
C:\dev\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2821: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set l

In [18]:
# Remove all profiles where at least one month is corrupted(contains null values in all columns)
corrupted = corrupted[~corrupted.ncodpers.isin(corrupted['ncodpers'][corrupted.age.isnull()])]
corrupted.drop('nomprov', axis=1, inplace=True) # Drop province name. It is 100% useless

In [19]:
X_train = corrupted.ix[:,:'segmento']

In [20]:
X_train.head()

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,ult_fec_cli_1t,indrel_1mes,tiprel_1mes,indresi,indext,conyuemp,canal_entrada,indfall,tipodom,cod_prov,ind_actividad_cliente,renta,segmento
0,2015-01-28,1375586,N,ES,H,35.0,2015-01-12,0.0,6.0,1.0,NaN,1,A,S,N,NaN,KHL,N,1.0,29.0,1.0,87218.10,02 - PARTICULARES
1,2015-01-28,1050611,N,ES,V,23.0,2012-08-10,0.0,35.0,1.0,NaN,1,I,S,S,NaN,KHE,N,1.0,13.0,0.0,35548.74,03 - UNIVERSITARIO
2,2015-01-28,1050612,N,ES,V,23.0,2012-08-10,0.0,35.0,1.0,NaN,1,I,S,N,NaN,KHE,N,1.0,13.0,0.0,122179.11,03 - UNIVERSITARIO
3,2015-01-28,1050613,N,ES,H,22.0,2012-08-10,0.0,35.0,1.0,NaN,1,I,S,N,NaN,KHD,N,1.0,50.0,0.0,119775.54,03 - UNIVERSITARIO
4,2015-01-28,1050614,N,ES,V,23.0,2012-08-10,0.0,35.0,1.0,NaN,1,A,S,N,NaN,KHE,N,1.0,50.0,1.0,NaN,03 - UNIVERSITARIO


In [21]:
y_train = corrupted.ix[:,'ind_ahor_fin_ult1':]
y_train.insert(0, 'ncodpers', X_train.ix[:,'ncodpers'])

In [22]:
y_train.head()

,ncodpers,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,1375586,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
1,1050611,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
2,1050612,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
3,1050613,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
4,1050614,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0


In [23]:
del corrupted

In [24]:
def sub_code_and_toint(text):
    return 0.0 if pd.isnull(text) else float(str(text)[1:2])
    
def preprocess_features(X):
    X.ind_empleado.replace(to_replace=[1.0, 99.0], value=[1.0, 0.0], inplace=True)
    
    X.indext.replace(to_replace=['S', 'N'], value=[1.0, 0.0], inplace=True)
    X.indresi.replace(to_replace=['S', 'N'], value=[1.0, 0.0], inplace=True)
    
    X.sexo.replace(to_replace=['V', 'H'], value=[1.0, 0.0], inplace=True)
    
    X.segmento = X.segmento.apply(sub_code_and_toint)

In [25]:
X_train.query('(fecha_dato > "2016-04-28")', inplace=True)
preprocess_features(X_train)

In [26]:
y_dict = {} #dict of user-id products for each month

def productMonthFraction(rows):
    labels = [rows.iloc[0]['ncodpers']]
    products = np.sum(rows.ix[:,1:], axis=0) / len(rows.iloc[:,0]) #only products flags
    labels.extend(products)

    y_dict[labels[0]] = labels

In [27]:
y_train.groupby('ncodpers').apply(productMonthFraction)

""


In [28]:
fraction = pd.DataFrame.from_dict(y_dict, orient='index')
fraction.columns = labels_names

In [29]:
fraction.head()

,ncodpers_y,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
15889.0,15889.0,0.0,0.0,1.000000,0.0,0.000000,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.470588,1.0,0.0,0.0,0.0,0.0
15890.0,15890.0,0.0,0.0,0.000000,0.0,1.000000,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,1.0,0.0,0.0,1.0,0.0,0.0,1.000000,0.0,0.0,1.0,1.0,1.0
15891.0,15891.0,0.0,0.0,0.500000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
15892.0,15892.0,0.0,0.0,0.705882,0.0,0.294118,0.0,0.0,0.0,0.0,0.0,0.0,0.882353,1.0,0.0,0.0,0.0,0.0,1.0,1.000000,1.0,0.0,0.0,0.0,1.0
15893.0,15893.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.117647,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.0,0.0,0.0,0.0,0.0


In [30]:
X_train = X_train.merge(fraction, left_on='ncodpers', right_on='ncodpers_y', how='inner')
X_train.drop('ncodpers_y', axis=1, inplace=True)
X_train.drop('fecha_dato', axis=1, inplace=True)
X_train.head()

,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,ult_fec_cli_1t,indrel_1mes,tiprel_1mes,indresi,indext,conyuemp,canal_entrada,indfall,tipodom,cod_prov,ind_actividad_cliente,renta,segmento,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,657640,N,ES,0.0,68.0,2006-11-17,0.0,114.0,1.0,NaN,1,I,1.0,0.0,NaN,KAQ,N,1.0,28.0,0.0,147347.97,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,657788,N,ES,0.0,42.0,2006-11-18,0.0,114.0,1.0,NaN,1,A,1.0,0.0,NaN,KBF,N,1.0,28.0,1.0,132559.35,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,657795,N,ES,1.0,44.0,2006-11-18,0.0,114.0,1.0,NaN,1,A,1.0,0.0,NaN,KAP,N,1.0,26.0,1.0,81399.57,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,657790,N,ES,1.0,42.0,2006-11-18,0.0,114.0,1.0,NaN,1,A,1.0,0.0,NaN,KBF,N,1.0,48.0,1.0,NaN,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.176471,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0
4,657794,N,ES,0.0,49.0,2006-11-18,0.0,114.0,1.0,NaN,1,A,1.0,0.0,NaN,KAT,N,1.0,8.0,1.0,102189.00,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
X_train.to_csv('../data/processed/train-processed.csv', index=False)